# Imports

In [10]:
import os
import numpy as np
import pydot
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from contextlib import redirect_stdout
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, \
ReLU, Add, MaxPool2D, GlobalAvgPool2D, Dense
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical
 

In [3]:
K = tf.keras.backend
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

## ResNet 50

### Using Functional API

### a) The Identity Block


The identity block is the standard block used in ResNets, and corresponds to the case where the input activation (say $a^{[l]}$) has the same dimension as the output activation (say $a^{[l+2]}$). To flesh out the different steps of what happens in a ResNet's identity block, here is an alternative diagram showing the individual steps

In [3]:
def identity_block(X, f, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # retrive filters 
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # first component of the main path
    X = tf.keras.layers.Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1),
                               padding = 'valid', name = conv_name_base + '2a',
                               kernel_initializer = tf.keras.initializers.GlorotUniform(seed=0))(X)
    
    X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = tf.keras.layers.Activation('relu')(X)
    
    # Second Component of the main path
    X = tf.keras.layers.Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1),
                               padding = 'same', name = conv_name_base + '2b',
                               kernel_initializer = tf.keras.initializers.GlorotUniform(seed=0))(X)
    
    X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = tf.keras.layers.Activation('relu')(X)
    
    # third component of the main path
    X = tf.keras.layers.Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1),
                               padding = 'valid', name = conv_name_base + '2c',
                               kernel_initializer = tf.keras.initializers.GlorotUniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
    
    # Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = tf.keras.layers.Add()([X, X_shortcut])
    X = tf.keras.layers.Activation('relu')(X)
    
    return X



### 2) Convolution Block
You can use this type of block when the input and output dimensions don't match up

In [4]:
def convolutional_block(X, f, filters, stage, block, s = 2):    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    
    X = tf.keras.layers.Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = tf.keras.initializers.glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = tf.keras.layers.Activation('relu')(X)
    
    
    X = tf.keras.layers.Conv2D(F2, (f, f), strides = (1,1), padding='same', name = conv_name_base + '2b', kernel_initializer = tf.keras.initializers.glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = tf.keras.layers.Activation('relu')(X)

    X = tf.keras.layers.Conv2D(F3, (1, 1), strides = (1,1), padding='valid', name = conv_name_base + '2c', kernel_initializer = tf.keras.initializers.glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    X_shortcut = tf.keras.layers.Conv2D(F3, (1, 1), strides = (s,s), padding='valid', name = conv_name_base + '1', kernel_initializer = tf.keras.initializers.glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Add shortcut value to main path
    X = tf.keras.layers.Add()([X, X_shortcut])
    X = tf.keras.layers.Activation('relu')(X)
    
    
    return X

### Building 50 layers ResNet

In [9]:
# GRADED FUNCTION: ResNet50

def ResNet50(input_shape=None, classes=None):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    try:
        # Define the input as a tensor with shape input_shape
        X_input = tf.keras.layers.Input(input_shape)


        # Zero-Padding
        X = tf.keras.layers.ZeroPadding2D((3, 3))(X_input)

        # Stage 1
        X = tf.keras.layers.Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = tf.keras.initializers.glorot_uniform(seed=0))(X)
        X = tf.keras.layers.BatchNormalization(axis = 3, name = 'bn_conv1')(X)
        X = tf.keras.layers.Activation('relu')(X)
        X = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(X)

        # Stage 2
        X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
        X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
        X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


        # Stage 3
        X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
        X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
        X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
        X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

        # Stage 4 
        X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
        X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
        X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
        X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
        X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
        X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

        # Stage 5
        X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
        X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
        X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

        # AVGPOOL
        X = tf.keras.layers.AveragePooling2D()(X)

        # output layer
        X = tf.keras.layers.Flatten()(X)
        X = tf.keras.layers.Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = tf.keras.initializers.glorot_uniform(seed=0))(X)


        # Create model
        model = tf.keras.models.Model(inputs = X_input, outputs = X, name='ResNet50')

        return model
    
    except ValueError:
        print("Please provide input_shape and classes")

In [10]:
model = ResNet50(input_shape = (224, 224, 3), classes = 1000)

In [10]:
def summary(output=None, target=None, model = None):
        """ Show / Save model structure (summary) """
        if model is not None:
            model.summary()
            if target is not None:
                os.makedirs(output, exist_ok=True)
                with open(os.path.join(output, target), "w") as f:
                    with redirect_stdout(f):
                        model.summary()

In [17]:
summary(output= './summary', target='summary', model = model)

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [18]:
def model_plot(model = None, file_name=None):
    if model and file_name is not None:
        print('Model plotting...')
        
        plot_model(model, to_file = file_name, show_shapes=True,
                   show_layer_names=True,
                   rankdir='LB', expand_nested=True, dpi=96)
        print('Model plotted..')

In [19]:
model_plot(model = model, file_name='ResNet_model.png')

Model plotting...
Model plotted..


## Another way

In [4]:
def conv_batchnorm_relu(x, filters, kernel_size, strides):
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,
               padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

In [5]:
def identity_block(tensor, filters):
    x = conv_batchnorm_relu(tensor, filters=filters, kernel_size=1, strides=1)
    x = conv_batchnorm_relu(x, filters=filters, kernel_size=3, strides=1)
    x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)  # notice: filters=4*filters
    x = BatchNormalization()(x)
 
    x = Add()([x, tensor])
    x = ReLU()(x)
    return x

In [6]:
def convolution_block(tensor, filters, strides):
    # left stream
    x = conv_batchnorm_relu(tensor, filters=filters, kernel_size=1, strides=strides) #[v]
    x = conv_batchnorm_relu(x, filters=filters, kernel_size=3, strides=1)
    x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)  # notice: filters=4*filters
    x = BatchNormalization()(x)
 
    # right stream
    shortcut = Conv2D(filters=4*filters, kernel_size=1, strides=strides)(tensor)  # notice: filters=4*filters
    shortcut = BatchNormalization()(shortcut)
 
    x = Add()([x, shortcut])
    x = ReLU()(x)
    return x

In [7]:
def resnet_block(x, filters, reps, strides):
    x = convolution_block(x, filters=filters, strides=strides)
    for _ in range(reps-1):
        x = identity_block(x, filters=filters)
    return x

## ResNet50

In [15]:
def resnet50(input_shape=None):
    if input_shape:
        input_data = Input(shape=input_shape)
        x = conv_batchnorm_relu(input_data, filters=64, kernel_size=7, strides=2)  # [3]: 7x7, 64, strides 2
        x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)  # [3]: 3x3 max mool, strides 2
        x = resnet_block(x, filters=64, reps=3, strides=1)
        x = resnet_block(x, filters=128, reps=4, strides=2)  # strides=2 ([2]: conv3_1)
        x = resnet_block(x, filters=256, reps=6, strides=2)  # strides=2 ([2]: conv4_1)
        x = resnet_block(x, filters=512, reps=3, strides=2)  # strides=2 ([2]: conv5_1)
 
        x = GlobalAvgPool2D()(x)  # [3]: average pool *it is not written any pool size so we use Global
 
        output = Dense(10, activation='softmax')(x)  # [3]: 1000-d fc, softmax
 
        model = Model(input_data, output, name='ResNet50')
    
        return model
        
    

In [13]:
model = resnet50(input_shape=(224, 224, 3))

In [14]:
summary(output= './summary', target='another_summary', model = model)

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 112, 112, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 112, 112, 64) 256         conv2d_53[0][0]                  
__________________________________________________________________________________________________
re_lu_49 (ReLU)                 (None, 112, 112, 64) 0           batch_normalization_53[0][0]     
___________________________________________________________________________________________

## Data

In [11]:
(train_x, train_y), (test_x, test_y) =  tf.keras.datasets.cifar10.load_data()

In [12]:
train_x = train_x.astype('float32')
test_x = test_x.astype('float32')

train_x = train_x/255.0
test_x = test_x/255.0

train_y = to_categorical(train_y)
test_y = to_categorical(test_y)


In [16]:
model = resnet50(input_shape = (32, 32, 3))

In [17]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [19]:
#model.fit(train_x, train_y, epochs=1, batch_size = 32)